# 00_Bronze: サインインログの取り込み

## 概要
このノートブックでは、Entra ID（Azure AD）のサインインログをCSVファイルから取り込み、Bronze層のテーブルを作成します。

## 処理フロー
1. カタログとスキーマの作成
2. CSV形式のままステージングテーブル（`signin_logs_bronze_src`）に取り込み
3. 派生列（タイムスタンプ変換、型変換など）を追加してBronze本番テーブル（`signin_logs_bronze`）を作成
4. パフォーマンス向上のため、日付でパーティショニング

## メダリオンアーキテクチャ - Bronze層
- **目的**: 生データを可能な限りそのまま保持し、データの完全性を確保
- **特徴**: 最小限の変換のみ実施（型変換、タイムスタンプ解析など）
- **利点**: データの再処理が容易、監査証跡の保持


## Step 1: カタログとスキーマの作成

Unity Catalogの階層構造（Catalog > Schema > Table）を準備します。


In [0]:
-- カタログ/スキーマ
CREATE CATALOG IF NOT EXISTS users;
CREATE SCHEMA  IF NOT EXISTS users.yukiteru_koide;
USE CATALOG users;
USE users.yukiteru_koide;


## Step 2: ステージングテーブルの作成

CSVファイルのスキーマをそのまま反映したステージングテーブルを作成します。
すべてのカラムを `STRING` 型で定義し、後続の処理で適切な型に変換します。

### テーブルスキーマ
- **id**: サインインイベントの一意識別子
- **createdDateTime**: イベント発生日時
- **userPrincipalName**: ユーザーのメールアドレス
- **appDisplayName**: アクセスしたアプリケーション名
- **ipAddress**: サインイン元のIPアドレス
- **locationCountryOrRegion / locationCity**: サインイン元の地理情報
- **statusErrorCode**: サインイン結果（0=成功、その他=エラーコード）


In [0]:
-- 1) 生CSVスキーマそのままのステージ表（取り込み専用）
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_logs_bronze_src (
  id STRING,
  createdDateTime STRING,
  userPrincipalName STRING,
  userId STRING,
  appDisplayName STRING,
  appId STRING,
  ipAddress STRING,
  clientAppUsed STRING,
  isInteractive STRING,
  authenticationRequirement STRING,
  conditionalAccessStatus STRING,
  deviceOperatingSystem STRING,
  browser STRING,
  locationCountryOrRegion STRING,
  locationCity STRING,
  statusErrorCode STRING,
  statusFailureReason STRING,
  statusAdditionalDetails STRING
) USING delta;


## Step 3: CSVファイルの取り込み

`COPY INTO` コマンドを使用して、Unity Catalog Volumeに配置されたCSVファイルをステージングテーブルに取り込みます。

### COPY INTOの特徴
- **冪等性**: 同じファイルを複数回実行しても重複しない
- **効率性**: 大量データの一括取り込みに最適
- **追跡性**: 取り込み履歴が自動的に記録される


In [0]:
-- 2) CSV → ステージ表（※ここは"パスを文字列で"）
COPY INTO users.yukiteru_koide.signin_logs_bronze_src
FROM 'dbfs:/Volumes/users/yukiteru_koide/security/entra_signin_logs.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true');


## Step 4: Bronze本番テーブルの作成

ステージングテーブルから本番Bronzeテーブルを作成します。この段階で以下の処理を実施：

### 実施する処理
1. **型変換**: 
   - `createdDateTime` → `TIMESTAMP` 型に変換
   - `statusErrorCode` → `INT` 型に変換
   - `isInteractive` → `BOOLEAN` 型に変換

2. **派生列の追加**:
   - `p_date`: パーティション用の日付カラム
   - `ingestion_ts`: データ取り込み時刻（監査用）
   - `source_path`: データソースのパス（トレーサビリティ）

3. **パーティショニング**:
   - 日付（`p_date`）でパーティション分割し、クエリパフォーマンスを向上


In [0]:
-- 3) 派生列付きの本番Bronze表（パーティション付き）
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_logs_bronze
USING delta
PARTITIONED BY (p_date) AS
SELECT
  *,
  to_timestamp(createdDateTime)                            AS created_ts,
  CAST(statusErrorCode AS INT)                             AS statusErrorCode_int,
  CASE lower(isInteractive)
    WHEN 'true' THEN true WHEN 'false' THEN false ELSE NULL END AS isInteractive_bool,
  to_date(to_timestamp(createdDateTime))                   AS p_date,
  current_timestamp()                                      AS ingestion_ts,
  '/Volumes/users/yukiteru_koide/security/entra_signin_logs.csv' AS source_path
FROM users.yukiteru_koide.signin_logs_bronze_src;


## Step 5: テーブルの最適化

`OPTIMIZE` コマンドでDelta Lakeテーブルの小さなファイルを結合し、クエリパフォーマンスを向上させます。

### OPTIMIZEの効果
- 小さなファイルを大きなファイルに結合（コンパクション）
- 読み取り時のI/O操作を削減
- クエリ実行速度の向上


In [0]:
OPTIMIZE users.yukiteru_koide.signin_logs_bronze;
